# 指标开发

In [22]:
%matplotlib inline
%matplotlib notebook

import vectorbtpro as vbt
import numpy as np
import pandas as pd

from numba import njit
import talib

# 管道优化
from vectorbtpro.returns import nb as ret_nb
from vectorbtpro.portfolio import nb as pf_nb
from vectorbtpro.portfolio.enums import Direction

vbt.settings.set_theme('dark')

# 数据读取

In [23]:
## Load m1 data
hdf_fname = "./data/EURUSD_M1_OHLCV_2021-01-01_2023-02-20_cleaned.h5"
data = vbt.HDFData.fetch(hdf_fname)
data.wrapper.index, data.wrapper.columns, data.symbols

(DatetimeIndex(['2021-01-03 22:00:00+00:00', '2021-01-03 22:01:00+00:00',
                '2021-01-03 22:02:00+00:00', '2021-01-03 22:03:00+00:00',
                '2021-01-03 22:04:00+00:00', '2021-01-03 22:05:00+00:00',
                '2021-01-03 22:06:00+00:00', '2021-01-03 22:07:00+00:00',
                '2021-01-03 22:08:00+00:00', '2021-01-03 22:09:00+00:00',
                ...
                '2023-02-19 23:50:00+00:00', '2023-02-19 23:51:00+00:00',
                '2023-02-19 23:52:00+00:00', '2023-02-19 23:53:00+00:00',
                '2023-02-19 23:54:00+00:00', '2023-02-19 23:55:00+00:00',
                '2023-02-19 23:56:00+00:00', '2023-02-19 23:57:00+00:00',
                '2023-02-19 23:58:00+00:00', '2023-02-19 23:59:00+00:00'],
               dtype='datetime64[ns, UTC]', name='time', length=793784, freq=None),
 Index(['Open', 'High', 'Low', 'Close', 'Volume'], dtype='object'),
 ['EURUSD_M1_Cleaned'])

In [24]:
data_m1_df = data.get(symbols="EURUSD_M1_Cleaned")
data_m1_df

,Open,High,Low,Close,Volume
time,,,,,
2021-01-03 22:00:00+00:00,1.224035,1.224035,1.223830,1.224005,30.160
2021-01-03 22:01:00+00:00,1.223915,1.224260,1.223905,1.224055,23.780
2021-01-03 22:02:00+00:00,1.224060,1.224070,1.223925,1.223925,17.030
2021-01-03 22:03:00+00:00,1.223935,1.224005,1.223815,1.223830,26.850
2021-01-03 22:04:00+00:00,1.223840,1.223880,1.223160,1.223535,50.675
...,...,...,...,...,...
2023-02-19 23:55:00+00:00,1.068625,1.068625,1.068495,1.068505,159.590
2023-02-19 23:56:00+00:00,1.068490,1.068540,1.068475,1.068510,114.095
2023-02-19 23:57:00+00:00,1.068515,1.068560,1.068485,1.068490,71.100


In [25]:
# 检查有无空值
data_m1_df[data_m1_df.isna().any(axis=1)]

,Open,High,Low,Close,Volume
time,,,,,


## 读取应用频段 
* `1H` 和 `1D` 数据 (直接调用df resampling 有未来)
* 重采样 `Resampling` + 对齐 `Alignment`

### 定义采样周期 (由大到小) `#5D` `#h` `#T`

In [26]:
sample_period_1 = "5D"
sample_period_2 = "1h"

### Resample 数据 `1T` 到 `1H` 和 `1D`

In [27]:
data_1D = data.resample(sample_period_1)
data_1h = data.resample(sample_period_2)

In [28]:
data_1D.get()

,Open,High,Low,Close,Volume
time,,,,,
2021-01-03 00:00:00+00:00,1.224035,1.234945,1.223110,1.226920,9.037053e+05
2021-01-08 00:00:00+00:00,1.226920,1.228465,1.213225,1.220875,5.655657e+05
2021-01-13 00:00:00+00:00,1.220890,1.222285,1.206620,1.206640,5.103144e+05
2021-01-18 00:00:00+00:00,1.206625,1.218960,1.205355,1.217060,7.019411e+05
2021-01-23 00:00:00+00:00,1.217090,1.218335,1.205860,1.210295,5.028639e+05
...,...,...,...,...,...
2023-01-28 00:00:00+00:00,1.086615,1.103315,1.080215,1.101280,9.476067e+05
2023-02-02 00:00:00+00:00,1.101270,1.102680,1.070945,1.072915,1.258081e+06
2023-02-07 00:00:00+00:00,1.072910,1.079085,1.066635,1.067740,1.607223e+06


In [29]:
data_1D.get(symbols="EURUSD_M1_Cleaned").iloc[:3]

,Open,High,Low,Close,Volume
time,,,,,
2021-01-03 00:00:00+00:00,1.224035,1.234945,1.223110,1.226920,903705.2556
2021-01-08 00:00:00+00:00,1.226920,1.228465,1.213225,1.220875,565565.7337
2021-01-13 00:00:00+00:00,1.220890,1.222285,1.206620,1.206640,510314.4330


In [30]:
data_1h.get(symbols="EURUSD_M1_Cleaned").iloc[:3]

,Open,High,Low,Close,Volume
time,,,,,
2021-01-03 22:00:00+00:00,1.224035,1.224260,1.223160,1.223435,1664.75505
2021-01-03 23:00:00+00:00,1.223415,1.225285,1.223110,1.225015,5449.76000
2021-01-04 00:00:00+00:00,1.225010,1.225010,1.223875,1.224080,7147.66000


### Resample 校验

In [31]:
data.get().iloc[[0,-1]]

,Open,High,Low,Close,Volume
time,,,,,
2021-01-03 22:00:00+00:00,1.224035,1.224035,1.22383,1.224005,30.16
2023-02-19 23:59:00+00:00,1.068420,1.068420,1.06836,1.068410,41.85


In [32]:
data_1D.get().iloc[[0,-1]]

,Open,High,Low,Close,Volume
time,,,,,
2021-01-03 00:00:00+00:00,1.224035,1.234945,1.223110,1.22692,903705.2556
2023-02-17 00:00:00+00:00,1.066550,1.069845,1.061285,1.06841,388130.9873


In [33]:
data_1h.get().iloc[[0, -1]]

,Open,High,Low,Close,Volume
time,,,,,
2021-01-03 22:00:00+00:00,1.224035,1.224260,1.223160,1.223435,1664.75505
2023-02-19 23:00:00+00:00,1.069115,1.069335,1.068255,1.068410,7560.48500


### 对齐数据

In [34]:
data_1h_open = data_1h.get(columns=["Open"]).dropna()
data_1h_close = data_1h.get(columns=["Close"]).dropna()
data_1h_high = data_1h.get(columns=["High"]).dropna()
data_1h_low = data_1h.get(columns=["Low"]).dropna()

# 预防未来加shift
data_1D_1h_open = data_1D.get(columns=["Open"]).resample(sample_period_2).first().ffill()
data_1D_1h_close = data_1D.get(columns=["Close"]).shift(1).resample(sample_period_2).last().ffill().shift(-1)
data_1D_1h_high = data_1D.get(columns=["High"]).shift(1).resample(sample_period_2).max().ffill().shift(-1)
data_1D_1h_low = data_1D.get(columns=["Low"]).shift(1).resample(sample_period_2).min().ffill().shift(-1)

In [35]:
data_1D_1h_open.size, data_1D_1h_close.size, data_1D_1h_high.size, data_1D_1h_low.size

(18601, 18601, 18601, 18601)

In [36]:
data_1h_open.size, data_1h_close.size, data_1h_high.size, data_1h_low.size

(13322, 13322, 13322, 13322)

In [37]:
h1_diff_1D_1h_idx = data_1h_open.index.difference(data_1D_1h_open.index)
D1_1h_diff_h1_idx = data_1D_1h_open.index.difference(data_1h_open.index)

In [38]:
data_1D_1h_open = data_1D_1h_open.drop(D1_1h_diff_h1_idx)
data_1D_1h_high = data_1D_1h_high.drop(D1_1h_diff_h1_idx)
data_1D_1h_low = data_1D_1h_low.drop(D1_1h_diff_h1_idx)
data_1D_1h_close = data_1D_1h_close.drop(D1_1h_diff_h1_idx)

In [39]:
data_1D_1h_open = pd.concat([data_1D_1h_open, 
                             pd.Series(index=h1_diff_1D_1h_idx, dtype=float).rename("Open").to_frame()], 
                             axis=0)
data_1D_1h_high = pd.concat([data_1D_1h_high, 
                             pd.Series(index=h1_diff_1D_1h_idx, dtype=float).rename("High").to_frame()], 
                             axis=0)
data_1D_1h_low = pd.concat([data_1D_1h_low, 
                            pd.Series(index=h1_diff_1D_1h_idx, dtype=float).rename("Low").to_frame()], 
                            axis=0)
data_1D_1h_close = pd.concat([data_1D_1h_close, 
                              pd.Series(index=h1_diff_1D_1h_idx, dtype=float).rename("Close").to_frame()], 
                              axis=0)

In [40]:
data_1D_1h_open.size, data_1D_1h_close.size, data_1D_1h_high.size, data_1D_1h_low.size

(13322, 13322, 13322, 13322)

In [41]:
# 校验对齐结果
h1_diff_1D_1h_idx = data_1h_open.index.difference(data_1D_1h_open.index)
D1_1h_diff_h1_idx = data_1D_1h_open.index.difference(data_1h_open.index)
h1_diff_1D_1h_idx.size, D1_1h_diff_h1_idx.size

(0, 0)

# 指标定义

## Super Trend

In [42]:
def get_basic_bands(med_price, atr, multiplier):
    matr = multiplier * atr
    upper = med_price + matr
    lower = med_price - matr
    return upper, lower

@njit
def get_final_bands_nb(close, upper, lower):
    trend = np.full(close.shape, np.nan)
    dir_ = np.full(close.shape, 1)
    long = np.full(close.shape, np.nan)
    short = np.full(close.shape, np.nan)

    for i in range(1, close.shape[0]):
        if close[i] > upper[i - 1]:
            dir_[i] = 1
        elif close[i] < lower[i - 1]:
            dir_[i] = -1
        else:
            dir_[i] = dir_[i - 1]
            if dir_[i] > 0 and lower[i] < lower[i - 1]:
                lower[i] = lower[i - 1]
            if dir_[i] < 0 and upper[i] > upper[i - 1]:
                upper[i] = upper[i - 1]

        if dir_[i] > 0:
            trend[i] = long[i] = lower[i]
        else:
            trend[i] = short[i] = upper[i]
            
    return trend, dir_, long, short

def faster_supertrend_talib(high, low, close, period=7, multiplier=3):
    avg_price = talib.MEDPRICE(high, low)
    atr = talib.ATR(high, low, close, period)
    upper, lower = get_basic_bands(avg_price, atr, multiplier)
    return get_final_bands_nb(close, upper, lower)

expr = """
    SuperTrend[st]:
    medprice = @talib_medprice(high, low)
    atr = @talib_atr(high, low, close, @p_period)
    upper, lower = get_basic_bands(medprice, atr, @p_multiplier)
    supert, superd, superl, supers = get_final_bands(close, upper, lower)
    supert, superd, superl, supers
"""

SuperTrend = vbt.IF.from_expr(
    expr, 
    takes_1d=True,
    get_basic_bands=get_basic_bands,
    get_final_bands=get_final_bands_nb,
    period=7, 
    multiplier=3
)

import plotly.express as px

class SuperTrend(SuperTrend):
    def get_df(self,
               column=None, 
               close_kwargs=None,
               superl_kwargs=None,
               supers_kwargs=None,
               fig=None, 
               **layout_kwargs):
        superl = self.select_col_from_obj(self.superl, column).rename('LongLine')
        supers = self.select_col_from_obj(self.supers, column).rename('ShortLine')
        superd = self.select_col_from_obj(self.superd, column).rename('Dir')
        supert = self.select_col_from_obj(self.supert, column).rename('Trend')
        
        return pd.concat([superl, supers, superd, supert], axis=1)
        
    def plot(self, 
             column=None, 
             close_kwargs=None,
             superl_kwargs=None,
             supers_kwargs=None,
             fig=None, 
             **layout_kwargs):
        close_kwargs = close_kwargs if close_kwargs else {}
        superl_kwargs = superl_kwargs if superl_kwargs else {}
        supers_kwargs = supers_kwargs if supers_kwargs else {}
        
        close = self.select_col_from_obj(self.close, column).rename('Close')
        supers = self.select_col_from_obj(self.supers, column).rename('Short')
        superl = self.select_col_from_obj(self.superl, column).rename('Long')
        
        fig = close.vbt.plot(fig=fig, **close_kwargs, **layout_kwargs)
        
        # 过滤 weekends
        # fig.update_xaxes(
        #     rangebreaks=[
        #         dict(bounds=["sat", "mon"]), #hide weekends
        #         # dict(values=["2015-12-25", "2016-01-01"])  # hide Christmas and New Year's
        #     ]
        # )
        
        supers.vbt.plot(fig=fig, **supers_kwargs)
        superl.vbt.plot(fig=fig, **superl_kwargs)
        
        return fig

In [43]:
PERIOD = 50
MULTIPLIER = 20

st = SuperTrend.run(
    high=data_1h_high, low=data_1h_low, close=data_1h_close, # 输入数据
    period=PERIOD, multiplier=MULTIPLIER, # 参数
    skipna=True,
    param_product=True,
    execute_kwargs=dict(show_progress=True)
)

st_mtf = st.get_df(column=(PERIOD, MULTIPLIER))
st_mtf.loc[slice("2022-05", "2022-06")]

  0%|          | 0/1 [00:00<?, ?it/s]

,LongLine,ShortLine,Dir,Trend
time,,,,
2022-05-01 21:00:00+00:00,NaN,1.088307,-1,1.088307
2022-05-01 22:00:00+00:00,NaN,1.088307,-1,1.088307
2022-05-01 23:00:00+00:00,NaN,1.088307,-1,1.088307
2022-05-02 00:00:00+00:00,NaN,1.088307,-1,1.088307
2022-05-02 01:00:00+00:00,NaN,1.088307,-1,1.088307
...,...,...,...,...
2022-06-30 19:00:00+00:00,NaN,1.074363,-1,1.074363
2022-06-30 20:00:00+00:00,NaN,1.074363,-1,1.074363
2022-06-30 21:00:00+00:00,NaN,1.074363,-1,1.074363


In [44]:
# 空值检查
st_mtf[st_mtf["Dir"].isna()]

,LongLine,ShortLine,Dir,Trend
time,,,,


## 日频 ATR 带

In [45]:
atr_band_mtf = vbt.talib("ATR", timeperiod = 20).run(
    close=data_1h_close, 
    high=data_1h_high,
    low=data_1h_low,
    skipna=True,
).real

atr_band_mtf.columns=["ATR_H1"]
atr_band_mtf

,ATR_H1
time,
2021-01-03 22:00:00+00:00,NaN
2021-01-03 23:00:00+00:00,NaN
2021-01-04 00:00:00+00:00,NaN
2021-01-04 01:00:00+00:00,NaN
2021-01-04 02:00:00+00:00,NaN
...,...
2023-02-17 19:00:00+00:00,0.001652
2023-02-17 20:00:00+00:00,0.001594
2023-02-17 21:00:00+00:00,0.001539


In [46]:
atr_band_mtf["1D_Open"] = data_1D_1h_open
atr_band_mtf

,ATR_H1,1D_Open
time,,
2021-01-03 22:00:00+00:00,NaN,1.224035
2021-01-03 23:00:00+00:00,NaN,1.224035
2021-01-04 00:00:00+00:00,NaN,1.224035
2021-01-04 01:00:00+00:00,NaN,1.224035
2021-01-04 02:00:00+00:00,NaN,1.224035
...,...,...
2023-02-17 19:00:00+00:00,0.001652,NaN
2023-02-17 20:00:00+00:00,0.001594,NaN
2023-02-17 21:00:00+00:00,0.001539,NaN


In [47]:
atr_band_mtf["atr_upper_band"] = 3 * atr_band_mtf["ATR_H1"] + atr_band_mtf["1D_Open"]
atr_band_mtf["atr_lower_band"] = -3 * atr_band_mtf["ATR_H1"] + atr_band_mtf["1D_Open"]
# atr_band_mtf[atr_band_mtf.isna().any(axis=1)]
atr_band_mtf

,ATR_H1,1D_Open,atr_upper_band,atr_lower_band
time,,,,
2021-01-03 22:00:00+00:00,NaN,1.224035,NaN,NaN
2021-01-03 23:00:00+00:00,NaN,1.224035,NaN,NaN
2021-01-04 00:00:00+00:00,NaN,1.224035,NaN,NaN
2021-01-04 01:00:00+00:00,NaN,1.224035,NaN,NaN
2021-01-04 02:00:00+00:00,NaN,1.224035,NaN,NaN
...,...,...,...,...
2023-02-17 19:00:00+00:00,0.001652,NaN,NaN,NaN
2023-02-17 20:00:00+00:00,0.001594,NaN,NaN,NaN
2023-02-17 21:00:00+00:00,0.001539,NaN,NaN,NaN


## 布林带

In [48]:
bbands_mtf = vbt.talib("BBANDS", timeperiod = 20).run(
    close=data_1h_close, 
    skipna=True,
)
bbands_mtf.lowerband
bbands_mtf.upperband
bbands_mtf.middleband

bbands_mtf = \
pd.concat([bbands_mtf.lowerband.rename(columns={"Close" : "lower_band"}), 
           bbands_mtf.middleband.rename(columns={"Close" : "middle_band"}), 
           bbands_mtf.upperband.rename(columns={"Close" : "upper_band"})
          ], axis=1)

bbands_mtf

,lower_band,middle_band,upper_band
time,,,
2021-01-03 22:00:00+00:00,NaN,NaN,NaN
2021-01-03 23:00:00+00:00,NaN,NaN,NaN
2021-01-04 00:00:00+00:00,NaN,NaN,NaN
2021-01-04 01:00:00+00:00,NaN,NaN,NaN
2021-01-04 02:00:00+00:00,NaN,NaN,NaN
...,...,...,...
2023-02-17 19:00:00+00:00,1.061196,1.065115,1.069034
2023-02-17 20:00:00+00:00,1.060944,1.065287,1.069631
2023-02-17 21:00:00+00:00,1.060785,1.065492,1.070200


# 指标组合显示

In [49]:
date_range = slice("2022-11-01", "2023-02-19")
data_1h.get().loc[date_range]
# data.get().loc[date_range]

,Open,High,Low,Close,Volume
time,,,,,
2022-11-01 00:00:00+00:00,0.988590,0.989030,0.988240,0.988545,8550.310
2022-11-01 01:00:00+00:00,0.988535,0.989990,0.988435,0.989495,17902.095
2022-11-01 02:00:00+00:00,0.989500,0.990815,0.989295,0.990650,14696.150
2022-11-01 03:00:00+00:00,0.990645,0.990780,0.989455,0.990350,14676.305
2022-11-01 04:00:00+00:00,0.990360,0.990790,0.989955,0.990520,12390.215
...,...,...,...,...,...
2023-02-19 19:00:00+00:00,NaN,NaN,NaN,NaN,NaN
2023-02-19 20:00:00+00:00,NaN,NaN,NaN,NaN,NaN
2023-02-19 21:00:00+00:00,NaN,NaN,NaN,NaN,NaN


In [50]:
date_range = slice("2023-01-01", "2023-05-08")

kwargs1 = {
    "title_text" : "H4 OHLCV with BBands on Price and RSI", 
    "title_font_size" : 18,
    "height" : 960,
    "width" : 1200,
    "legend" : dict(yanchor="top", y=1.0, xanchor="left", x= 0.1)
}

fig = vbt.make_subplots(rows=2,cols=1, shared_xaxes=True, vertical_spacing=0.1)

fig = data_1h.get().loc[date_range, ["Open", "High", "Low", "Close", "Volume"]].vbt.ohlcv.plot(add_trace_kwargs=dict(row=1, col=1), fig=fig, **kwargs1)
fig = atr_band_mtf.loc[date_range, ["atr_upper_band"]].vbt.plot(add_trace_kwargs=dict(row=1, col=1), fig=fig, **kwargs1)
fig = atr_band_mtf.loc[date_range, ["atr_lower_band"]].vbt.plot(add_trace_kwargs=dict(row=1, col=1), fig=fig, **kwargs1)
# fig = bbands_mtf.loc[date_range, ["lower_band"]].vbt.plot(add_trace_kwargs=dict(row=1, col=1), fig=fig, **kwargs1)
# fig = bbands_mtf.loc[date_range, ["upper_band"]].vbt.plot(add_trace_kwargs=dict(row=1, col=1), fig=fig, **kwargs1)
# fig = bbands_mtf.loc[date_range, ["middle_band"]].vbt.plot(add_trace_kwargs=dict(row=1, col=1), fig=fig, **kwargs1)
fig = st_mtf.loc[date_range, ["LongLine"]].vbt.plot(add_trace_kwargs=dict(row=1, col=1), fig=fig, **kwargs1)
fig = st_mtf.loc[date_range, ["ShortLine"]].vbt.plot(add_trace_kwargs=dict(row=1, col=1), fig=fig, **kwargs1)

fig#.show_svg()

FigureWidget({
    'data': [{'close': array([    nan,     nan,     nan, ...,     nan, 1.0691 , 1.06841]),
              'decreasing': {'fillcolor': '#ee534f', 'line': {'color': '#ee534f'}},
              'high': array([     nan,      nan,      nan, ...,      nan, 1.06938 , 1.069335]),
              'increasing': {'fillcolor': '#26a69a', 'line': {'color': '#26a69a'}},
              'low': array([     nan,      nan,      nan, ...,      nan, 1.068725, 1.068255]),
              'name': 'OHLC',
              'opacity': 0.75,
              'open': array([     nan,      nan,      nan, ...,      nan, 1.0689  , 1.069115]),
              'type': 'candlestick',
              'uid': '5f555fe6-db86-4725-bd3c-8925b41f9a68',
              'x': array([datetime.datetime(2023, 1, 1, 0, 0, tzinfo=<UTC>),
                          datetime.datetime(2023, 1, 1, 1, 0, tzinfo=<UTC>),
                          datetime.datetime(2023, 1, 1, 2, 0, tzinfo=<UTC>), ...,
                          datetime.datetime(

# 生成指标序列

In [51]:
atr_band_mtf.shape[0], st_mtf.shape[0], bbands_mtf.shape[0]

(13322, 13322, 13322)

In [52]:
total_mtf = pd.concat([atr_band_mtf, st_mtf, bbands_mtf], 
                       keys=["atr_band", "st", "bbands"], axis=1)
total_mtf.columns

MultiIndex([('atr_band',         'ATR_H1'),
            ('atr_band',        '1D_Open'),
            ('atr_band', 'atr_upper_band'),
            ('atr_band', 'atr_lower_band'),
            (      'st',       'LongLine'),
            (      'st',      'ShortLine'),
            (      'st',            'Dir'),
            (      'st',          'Trend'),
            (  'bbands',     'lower_band'),
            (  'bbands',    'middle_band'),
            (  'bbands',     'upper_band')],
           )

In [53]:
total_mtf.to_hdf("./data/indicator_family.h5", "EURUSD_1h_1D_indicator_family")